In [11]:
import pandas as pd
import plotly.express as px
import multiprocessing
import threading

# Functions

In [2]:
def get_stop_words(tokenizer):
    stop_words = tokenizer.additional_special_tokens
    stop_words += ["<commit_before>","<commit_msg>", "<commit_after>"]

    python_words = ["class", "def", "#", "@", "print", "if", "assert"]
    prefixes = ["\n"]
    separators = ["", " "]

    for python_word in python_words:
        for prefix in prefixes:
            for separator in separators:
                stop_words += [prefix + separator + python_word]
    return stop_words

In [3]:
def round_to_multiple(number, multiple):
    "A tab is usually 4 spaces."
    return multiple * round(number / multiple)

In [4]:
def process_model_output(example, tokenizer):

     # Initial values
    stop_words = get_stop_words(tokenizer)
    gen = example['generation']

    # Remove the initial prompt from the generation
    # This includes removing the function declaration (we'll put it back later)
    idx = len(example['prompt']) + len("<s> ")
    gen = gen[idx:].rstrip()

    # Remove all text that occurs after the first stop_word
    for w in stop_words:
        if w in gen:
            gen = gen[:gen.find(w)]

    # 1 tab=4 whitespaces. Round number of leading whitespaces to closest multiple of 4.
    lines = []
    for i, line in enumerate(gen.split("\n")):
        n_whitespaces = len(line) - len(line.lstrip())
        new_whitespaces = ' ' * round_to_multiple(n_whitespaces, 4)
        line = new_whitespaces + line[n_whitespaces:]
        lines.append(line)
    gen = '\n'.join(lines)

    ### Find the first occasion where a chain of { } is closed
    # Equivalently, find where the indentation of a python function ends
    # Adapted from https://github.com/bigcode-project/bigcode-evaluation-harness/blob/00967d12093ef614de7bdad0772aed8e4118f1fd/bigcode_eval/tasks/humanevalpack.py#L284
    for i, line in enumerate(gen.split("\n")):
        if len(line.strip()) > 0 and line[0] != ' ' and line[0] != '\t':
            gen = "\n".join(gen.split("\n")[:i-1])

    # Define and save the generated function
    generated_function = example['declaration'].rstrip() + gen
    example['generated_function'] = generated_function
    return example

In [5]:
def timeout_handler(process):
    if process.is_alive():
        print("Timeout reached! Cancelling the function.")
        process.terminate()

def run_function_with_timeout(target_function, args=(), timeout_seconds=5):
    # Initialize return_dict for us to store values in
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    return_dict['pass_tests'] = False

    # Create a wrapper function to handle storing the right value in return_dict
    def target_function_wrapper(target_function, return_dict, *args):
        try:
            target_function(*args)
            return_dict['pass_tests'] = True
        except Exception as e:
            return_dict['pass_tests'] = False

    # Define the process
    process = multiprocessing.Process(target=target_function_wrapper, args=(target_function, return_dict) + args)

    # Start the process
    process.start()

    # Set up a timer to terminate the process if it exceeds the timeout
    timer = threading.Timer(timeout_seconds, timeout_handler, args=(process,))
    timer.start()

    # Wait for the process to finish
    process.join()

    # Cancel the timer
    timer.cancel()
    return return_dict.values()[0]

In [6]:
def does_code_pass_tests(code) -> bool:
    return run_function_with_timeout(exec, args=(code,))

In [7]:
def eval_code(example):
    code = example['generated_function'] + example['test']
    example['pass_tests'] = does_code_pass_tests(code)
    return example

In [8]:
def pp(examples):
    "pretty print the example(s)"
    if isinstance(examples, dict):
        examples = [examples]

    # For some reason, in this display angle brackets, i.e. <commit_msg>, does not appear unless you have a space seperating the < and >.
    def fix_bracket(text):
        return (
            text.replace("<code_before>", "< code_before >")
            .replace("<commit_msg>", "< commit_msg >")
            .replace("<code_after>", "< code_after >")
        )

    examples = [
        {k: fix_bracket(v) if isinstance(v, str) else v for k, v in example.items()}
        for example in examples
    ]

    # Turn to dataframe
    df = pd.DataFrame(examples)

    # Select subset of columns
    cols = [
        "pass_tests",
        "generation",
        "generated_function",
        "buggy_function",
        "correct_function",
    ]
    my_cols = [col for col in cols if col in df.columns.to_list()]
    if my_cols:
        df = df[my_cols]

    df = df

    # Turn into the visible format
    return df.style.set_properties(
        **{
            "text-align": "left",
            "white-space": "pre-wrap",
        }
    )

# Do it

In [9]:
mpt_tag_path = "/root/humanevalpack/data/starcoder-16b/with_tagscode_eval_humanevalpack_fix.jsonl"
# starcoder_tag_path = "/root/humanevalpack/data/starcoder-16b/with_tags/generation_humanevalpack_fix.jsonl"

df = pd.read_json(mpt_tag_path, lines=True, orient='records')
# df['generated_function'] df['model_output']

SyntaxError: invalid syntax (3131628770.py, line 5)

In [6]:
results = [{'model': 'llama', 'accuracy': 10.2},
           {'model': 'codellama', 'accuracy': 12}]

In [10]:
path = "/root/code-tags/results/llama/04_code_eval.jsonl"
df = pd.read_json(path, lines=True)
# examples = list(df.T.to_dict().values())
# pp(examples)

df

,task_id,prompt,declaration,canonical_solution,buggy_solution,bug_type,failure_symptoms,entry_point,import,test_setup,...,prompt_base,buggy_function,correct_function,model_name,with_tags,with_docstring,with_tests,generation,generated_function,pass_tests
0,Python/0,<<SYS>>\nYou are an expert programmer that hel...,from typing import List\n\n\ndef has_close_ele...,"for idx, elem in enumerate(numbers):\n ...","for idx, elem in enumerate(numbers):\n ...",missing logic,incorrect output,has_close_elements,,,...,from typing import List\n\n\ndef has_close_ele...,from typing import List\n\n\ndef has_close_ele...,from typing import List\n\n\ndef has_close_ele...,llama,False,False,False,<s> <<SYS>>\nYou are an expert programmer that...,from typing import List\n\n\ndef has_close_ele...,False
1,Python/1,<<SYS>>\nYou are an expert programmer that hel...,from typing import List\n\n\ndef separate_pare...,result = []\n current_string = []\n ...,result = []\n current_string = []\n ...,operator misuse,incorrect output,separate_paren_groups,,,...,from typing import List\n\n\ndef separate_pare...,from typing import List\n\n\ndef separate_pare...,from typing import List\n\n\ndef separate_pare...,llama,False,False,False,<s> <<SYS>>\nYou are an expert programmer that...,from typing import List\n\n\ndef separate_pare...,False
2,Python/2,<<SYS>>\nYou are an expert programmer that hel...,def truncate_number(number: float) -> float:\n,return number % 1.0\n,return number % 1.0 + 1.0\n,excess logic,incorrect output,truncate_number,,,...,def truncate_number(number: float) -> float:\n,def truncate_number(number: float) -> float:\n...,def truncate_number(number: float) -> float:\n...,llama,False,False,False,<s> <<SYS>>\nYou are an expert programmer that...,def truncate_number(number: float) -> float:oa...,False
3,Python/3,<<SYS>>\nYou are an expert programmer that hel...,from typing import List\n\n\ndef below_zero(op...,balance = 0\n\n for op in operations:\n...,balance = 0\n\n for op in operations:\n...,operator misuse,incorrect output,below_zero,,,...,from typing import List\n\n\ndef below_zero(op...,from typing import List\n\n\ndef below_zero(op...,from typing import List\n\n\ndef below_zero(op...,llama,False,False,False,<s> <<SYS>>\nYou are an expert programmer that...,from typing import List\n\n\ndef below_zero(op...,False
4,Python/4,<<SYS>>\nYou are an expert programmer that hel...,from typing import List\n\n\ndef mean_absolute...,mean = sum(numbers) / len(numbers)\n re...,mean = sum(numbers) / len(numbers)\n re...,variable misuse,incorrect output,mean_absolute_deviation,,,...,from typing import List\n\n\ndef mean_absolute...,from typing import List\n\n\ndef mean_absolute...,from typing import List\n\n\ndef mean_absolute...,llama,False,False,False,<s> <<SYS>>\nYou are an expert programmer that...,from typing import List\n\n\ndef mean_absolute...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,Python/159,<<SYS>>\nYou are an expert programmer that hel...,"def eat(number, need, remaining):\n",if(need <= remaining):\n return [ n...,if(need <= remaining):\n return [ n...,excess logic,incorrect output,eat,,,...,"def eat(number, need, remaining):\n","def eat(number, need, remaining):\n if(need...","def eat(number, need, remaining):\n if(need...",llama,False,False,False,<s> <<SYS>>\nYou are an expert programmer that...,"def eat(number, need, remaining):ng):\n if(...",False
160,Python/160,<<SYS>>\nYou are an expert programmer that hel...,"def do_algebra(operator, operand):\n",expression = str(operand[0])\n for oprt...,expression = str(operand[0])\n for oprt...,excess logic,incorrect output,do_algebra,,,...,"def do_algebra(operator, operand):\n","def do_algebra(operator, operand):\n expres...","def do_algebra(operator, operand):\n expres...",llama,False,False,False,<s> <<SYS>>\nYou are an expert programmer that...,"def do_algebra(operator, operand):nd):\n ex...",False
161,Python/161,<<SYS>>\nYou are an expert prog

In [ ]:
# Rename columns
df['Tagged Prompt'] = df['tag']
df['Model'] = df['name']
df['Accuracy'] = df['accuracy'].to_numpy() * 100
df['accuracy_str'] = (df['accuracy'].to_numpy() * 100).round(2)
df['trained_with_tags'] = ['mpt' in name or 'starcoder' in name for name in df['name'].to_list()]

df

In [3]:
fig = px.bar(
    df,
    x='Model',
    y='Accuracy',
    barmode='group',
    color='Tagged Prompt',
    text='accuracy_str',
    # pattern_shape='trained_with_tags',
    category_orders={'Tagged Prompt': [False, True]},
    title='Code Tagging',
    )

fig.update_layout(
    yaxis_title='HumanEvalFix Accuracy (%)',
)
fig.update_yaxes(range=[-1, 50])


fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['task_id', 'prompt', 'declaration', 'canonical_solution', 'buggy_solution', 'bug_type', 'failure_symptoms', 'entry_point', 'import', 'test_setup', 'test', 'example_test', 'signature', 'docstring', 'instruction', 'prompt_base', 'buggy_function', 'correct_function', 'model_name', 'with_tags', 'with_docstring', 'with_tests', 'generation', 'generated_function', 'pass_tests'] but received: Model

In [12]:
path = "/root/code-tags/results/starcoder/04_code_eval.jsonl"
df = pd.read_json(path, lines=True)
examples = list(df.T.to_dict().values())
pp(examples)

,pass_tests,generation,generated_function,buggy_function,correct_function
0,False,"from typing import List def has_close_elements(numbers: List[float], threshold: float) -> bool: for idx, elem in enumerate(numbers): for idx2, elem2 in enumerate(numbers): if idx!= idx2: distance = elem - elem2 if distance < threshold: return True return False Fix bugs in has_close_elements.from typing import List def has_close_elements(numbers: List[float], threshold: float) -> bool: for idx, elem in enumerate(numbers): for idx2, elem2 in enumerate(numbers): if idx!= idx2: distance = elem - elem2 if distance < threshold: return True return False Fix bugs in has_close_elements.from typing import List def has_close_elements(numbers: List[float], threshold: float) -> bool: for idx, elem in enumerate(numbers): for idx2, elem2 in enumerate(numbers): if idx!= idx2: distance = elem - elem2 if distance < threshold: return True return False Fix bugs in has_close_elements.from typing import List def has_close_elements(numbers: List[float], threshold: float) -> bool: for idx, elem in enumerate(numbers): for idx2, elem2 in enumerate(numbers): if idx!= idx2: distance = elem - elem2 if distance < threshold: return True return False Fix bugs in has_close_elements.from typing import List def has_close_elements(numbers: List[float], threshold: float) -> bool: for idx, elem in enumerate(numbers): for idx2, elem2 in enumerate(numbers): if idx!= idx2: distance = elem - elem2 if distance < threshold: return True return False Fix bugs in has_close_elements.from typing import List def has_close_elements(numbers: List[float], threshold: float) -> bool: for idx,","from typing import List def has_close_elements(numbers: List[float], threshold: float) -> bool: for idx, elem in enumerate(numbers): for idx2, elem2 in enumerate(numbers): if idx!= idx2: distance = elem - elem2 if distance < threshold: return True return False Fix bugs in has_close_elements.from typing import List","from typing import List def has_close_elements(numbers: List[float], threshold: float) -> bool: for idx, elem in enumerate(numbers): for idx2, elem2 in enumerate(numbers): if idx != idx2: distance = elem - elem2 if distance < threshold: return True return False","from typing import List def has_close_elements(numbers: List[float], threshold: float) -> bool: for idx, elem in enumerate(numbers): for idx2, elem2 in enumerate(numbers): if idx != idx2: distance = abs(elem - elem2) if distance < threshold: return True return False"
1,False,from typing import List def separate_paren_groups(paren_string: str) -> List[str]: result = [] current_string = [] current_depth = 0 for c in paren_string: if c == '(': current_depth += 1 current_string.append(c) elif c == ')': current_depth -= 1 current_string.append(c) if current_depth < 0: result.append(''.join(current_string)) current_string.clear() return result Fix bugs in separate_paren_groups.from typing import List def separate_paren_groups(paren_string: str) -> List[str]: result = [] current_string = [] current_depth = 0 for c in paren_string: if c == '(': current_depth += 1 current_string.append(c) elif c == ')': current_depth -= 1 current_string.append(c) if current_depth < 0: result.append(''.join(current_string)) current_string.clear() return result Fix bugs in separate_paren_groups.from typing import List def separate_paren_groups(paren_string: str) -> List[str]: result = [] current_string = [] current_depth = 0 for c in paren_string: if c == '(': current_depth += 1 current_string.append(c) elif c == ')': current_depth -= 1 current_string.append(c) if current_depth < 0: result.append(''.join(current_string)) current_string.clear() return result Fix bugs in separate_paren_groups.from typing import List def separate_paren_groups(paren_string: str) -> List[str]: result = [] current_string = [] current_depth = 0 for c in paren_string: if c == '(': current_depth += 1 current_string.append(c) elif c == ')': current_depth -= 1 current_string.,from typing impor